# hide
title: There's a new parking scraper in town
enable: plotly

... just, that new scraper is [mine]({% post_url 2021/2021-04-08-one-year-parking %}). There are people doing this some years longer! So let me instead introduce [parkendd.de](https://parkendd.de/) by [Offenes Dresden](https://github.com/offenesdresden). It's [all open](https://github.com/offenesdresden/ParkAPI/) and the archive is available [for download](https://parkendd.de/dumps/).

This post is similar to some of my other data investigation posts in the sense that i simply start coding and see what comes out of it. For a change, all code is included so you do not need to check the jupyter notebook. 

Currently (end of 2021) data from 2015 to 2020 is packaged into a big tar.xz file, which i will convert to tar.gz because it's easier to read in python. 

```
wget https://parkendd.de/dumps/Archive.tar.xz
xz -dc Archive.tar.xz | gzip -cf9 > parkapi-2020.tar.gz
```

The xz compression actually seems to be a good choice because the filesize expands from 200 to 500 megabytes with gz. Not a problematic number, though. However, the uncompressed tar file is about 3.6 Gigabytes. I want to use [pandas](https://pandas.pydata.org) and experience tells me that loading a Gigabyte csv will usually not fit into memory. Even if it does, all operations that *copy* data will eventually kill the python kernel. 

So, i'll iterate through all files in the archive - each representing one parking lot per year - resample them to averaged **1 hour buckets** and gradually merge them into a single DataFrame. I want to look at years 2016 to 2020, so that's about 44,000 hour steps for a 100+ paring lots which should fit into anyone's memory. 

In [ ]:
from pathlib import Path
import tarfile
import codecs
import re
from typing import Generator, Tuple, Union, Optional, Callable

from tqdm import tqdm
import requests
import pandas as pd
import numpy as np
import plotly
import plotly.express as px
from plotly.subplots import make_subplots

pd.options.display.max_columns = 30
pd.options.plotting.backend = "plotly"
plotly.templates.default = "plotly_dark"

In [ ]:
def iter_archive_dataframes(
    filename: Union[str, Path],
    resampling: str = "1h",
) -> Generator[Tuple[str, pd.DataFrame], None, None]:
    
    # tarfile does handle the gzip automatically
    with tarfile.open(filename) as tfp:
        
        # build map of lot_id to available csv filenames 
        #   i ignore 2015 since it's incomplete
        lot_id_filenames = dict()
        for filename in sorted(tfp.getnames()):
            if "backup" not in filename:
                match = re.match("(.*)-(20\d\d).csv", filename)
                if match:
                    lot_id, year = match.groups()
                    if year != "2015":
                        lot_id_filenames.setdefault(lot_id, []).append(filename)
        
        # for each lot
        for lot_id, filenames in lot_id_filenames.items():
            # if we have years 2016 - 2020
            if len(filenames) == 5:
                # build one DataFrame, resampled to 1 hour
                dfs = []
                for filename in filenames:
                    fp = tfp.extractfile(filename)
                    dfs.append(pd.read_csv(
                        codecs.getreader("utf-8")(fp), 
                        names=["date", "free"]
                    ))
                df = pd.concat(dfs, axis=0)
                df["date"] = pd.to_datetime(df["date"])
                try:
                    df = df.set_index("date").resample(resampling).mean()
                    yield lot_id, df
                except:
                    pass

In [ ]:
archive_file = Path("~/prog/data/parking/parkapi-2020.tar.gz").expanduser()
table_file = Path("~/prog/data/parking/parkapi-2020-1h.csv").expanduser()

if not table_file.exists():
    big_df = None
    for lot_id, df in tqdm(iter_archive_dataframes(archive_file)):
        df["lot_id"] = lot_id
        df = df.reset_index().set_index(["date", "lot_id"])
        if big_df is None:
            big_df = df
        else:
            # append rows and sort by date
            big_df = pd.concat([big_df, df]).sort_index()
    
    # x = lot_id, y = date
    big_df = big_df.unstack("lot_id")
    # drop the "free" label from columns, just keep lot_id
    big_df.columns = big_df.columns.droplevel()
    # store
    big_df.to_csv(table_file)

else:
    # read the file if it was already created
    big_df = pd.read_csv(table_file)
    big_df["date"] = pd.to_datetime(big_df["date"])
    big_df.set_index("date", inplace=True)
    big_df.columns.name = "lot_id"
    
big_df

Don't mind all the *NaN*s, the city of Aalborg is not scraped throughout the whole period. But Oldenburg seems to look good. Without further number crunching let's do a quick interactive plot, resampled to 1 week buckets:

In [ ]:
(big_df
 .resample("1w").mean()
 .round()  # the round saves about 300 Kb of javascript code 
 .plot(
     title=f"average number of free spaces per week ({big_df.shape[1]} lots)", 
     labels={"value": "number of free spaces", "date": "week"}
 )
)

As usual, you can drag and zoom, and hide the inidividual lots on the right side (doubleclick to hide all except one).

Now that i'm actually able to look at parking data predating this stupid covid pandemic i'll pose two simple research questions
- *Is the lockdown around Germany in beginning of 2020 visible in the parking lot occupation data?* 
- *Has anything in the parking behaviour significantly changed compared to before?* 

First of all, when checking the plots above, a few cities have big chunks of missing data, Aalborg for example. It's a shame but i'll exclude them. Moreover, there are smaller gaps. Sometimes it happens that the number of free spaces listed on a website gets stuck, or is not listed at all, while other lots on the same site work fine. I'll count the number of times that the average value does not change during three days. Specifically since year 2018:

In [ ]:
df = (
    big_df[(big_df.index >= "2018-01-01")]
    .resample("1d").mean()
    .replace(np.nan, 0)  # treat missing values as zero
)
num_equal_days = ((df == df.shift(1)) & (df == df.shift(2))).astype(int).sum()
num_equal_days.sort_values().plot.bar(
    title="Number of times that 3 consecutive days have unchanged number of free spaces",
    height=600,
)

By visual inspection and comparison with the plot on top i decide to cut everything above 100, and also remove the Zurich lot because it misses data exactly at the time in question:

In [ ]:
big_df = big_df.loc[:, (num_equal_days <= 100) & (big_df.columns != "zuerichparkgarageamcentral")]
big_df.shape

Okay, 53 lots remain. Now it would be great to *normalize* each lot using the total capacity.

In [ ]:
big_df.max()

Ah, well, the congress center in Dresden probably does not had 26 thousand spaces. I'll first clamp the dataframe to, let's say, 2000, just to remove the most obvious outliers

In [ ]:
big_df = big_df.clip(0, 2000)

and then ask the ParkAPI for more precise values. The endpoint is `https://api.parkendd.de/<City>` which returns static and live data for each lot per city:

In [ ]:
CITIES = ["Aarhus", "Dresden", "Freiburg", "Ingolstadt", "Luebeck"]
lot_infos = dict()
for city in CITIES:
    response = requests.get(f"https://api.parkendd.de/{city}")
    for lot in response.json()["lots"]:
        lot["city"] = city
        lot_infos[lot["id"]] = lot

lot_infos["dresdenkongresszentrum"]

Well 26,000 was only two magnitudes above the truth.

In [ ]:
lot_infos["luebeckbackbord"]

Lübeck does not provide a total value. The website that is scraped can be determined from the [geojson file](https://raw.githubusercontent.com/offenesdresden/ParkAPI/master/park_api/cities/Luebeck.geojson) of the Lübeck-scraper (or from `https://api.parkendd.de/`). It actually seems to be [offline](https://www.kwl-luebeck.de/parken/aktuelle-parkplatzbelegung) right now. So i'll use the official numbers if present and the maximum free value otherwise:

In [ ]:
official_capacity = pd.Series(
    big_df.columns.map(lambda c: lot_infos[c]["total"] or None), 
    index=big_df.columns
).dropna()

capacity = big_df.max()
capacity[official_capacity.index] = official_capacity

# lot occupation in range [0, 1]
occupied = 1. - (big_df / capacity).clip(0, 1)

(occupied
 .groupby(lambda c: lot_infos[c]["city"], axis=1).mean()
 .resample("1m").mean() * 100.
).plot(
    title="Average lot occupation per month and city",
    labels={"value": "occupation percentage", "date": "month"}
)

Alright. There it is. A pretty obvious dent! With the least occupation during April 2020. That's how i remember it. Kids skating on empty parking lots, no planes in the sky, no stupid shops selling useless things.

For the interested, here's the same plot for each lot:

In [ ]:
(occupied.resample("1m").mean() * 100.).round().plot(
    title="Average lot occupation per month and lot",
    labels={"value": "occupation percentage", "date": "month"}
)

There are more ways of looking at the occupation data. Instead of calculating the average for each week we can build a histogram of the occupation values. This shows all levels of occupation during each week:

In [ ]:
def plot_histogram(
        df: pd.DataFrame, 
        resample: str = "1w", 
        bins: int = 48, 
        range: Optional[Tuple[float, float]] = None,
        clip: Optional[Tuple[float, float]] = None,
        title: Optional[str] = None,
        labels: Optional[dict] = None,
):
    if range is None:
        df_n = df.replace(np.nan, 0)
        range = (np.amin(df_n.values), np.amax(df_n.values))
    df = pd.concat(
        (pd.Series(np.histogram(group, bins=bins, range=range)[0], name=key)
        for key, group in df.resample(resample, level="date")),
        axis=1
    ).replace(0, np.nan)
    df.index = np.linspace(*range, bins)
    if clip is not None:
        df = df.clip(*clip)
    return px.imshow(
        df, origin="lower",
        title=title or "Weekly histogram of occupation per lot",
        labels=labels or {"y": "occupation percentage", "x": "week"},
        color_continuous_scale=["#005", "#08f", "#8ff", "#fff", "#fff"]
    )

# ignore values that are exactly zero or one
#   as they are usually *bad data* (see below)
plot_histogram(occupied.replace({0: np.nan, 1: np.nan}) * 100.)

So, starting end of March 2020, the most reported lot occupation is between 0 and 15%. The situation kind of normalizes in June and kind of returns in November. 

What are these small horizontal stripes you ask? And what happened in the beginning of 2018? 

The short 2018 outage is probably some internal server problem. You know, disk full, provider problems. There is no indication in the [commit history](https://github.com/offenesdresden/ParkAPI/commits/master?after=75ad91d24cb709c2e2065e2ff48a1628973926b9+174&branch=master).  

To investigate the stripes, i'll spend few more Megabytes of generated javascript and look at a few lots in pariticular:

In [ ]:
def plot_lot_data(lot_id: str, filter: Optional[Callable] = None):
    fig = make_subplots(
        rows=2, cols=1,
        vertical_spacing=0.1,
        shared_xaxes=True,
        subplot_titles=["weekly occupation histogram", "number of free spaces per hour"],
    )
    filter = filter or (lambda df: df)
    df = filter(occupied[lot_id])
    histo = plot_histogram(df * 100)
    fig.add_trace(histo.data[0], row=1, col=1)
    fig.add_trace(
        filter(big_df[lot_id]).round().plot().data[0].update(showlegend=False), 
        row=2, col=1,
    )
    return fig.update_layout(
        coloraxis=histo.layout.coloraxis, 
        title=f"{lot_id} (capacity: {lot_infos[lot_id]['total']})", height=700
    )

plot_lot_data("dresdenparkhausmitte")

Obviously, a horizontal stripe means that the free-spaces-counter stood still somehow. Except for the stripes at 0% occupation starting at the end of 2019. They are caused by the reported number of free spaces being larger than the reported lot capacity, which is (by the time of writing this article) 280. This garage must have decreased it's capacity in the meantime. It would be helpful if the recorded capacity would be published in the archive as well. Otherwise we must trust the maximum value which is 432 for this recording. However, if you zoom in at Oct 1st to 4th 2016 when this maximum was reached, you'll notice a completely unrealistic looking period of 200+ spaces. Also note that the little free peaks that occur each day around that period are upside-down within! It may still be possible that some real-life event has caused that but i find it more likely to be some digital mess-up. 

In [ ]:
plot_lot_data("freiburgambahnhof")

At first glance, the parking lot in Freibug looks much more lively compared to the one above. But please zoom in at the flat-line in winter 2016/17. There is obviously no real car activity but still the number of reported free spaces changes between zero and 62 each day in a super regular pattern reminding on opening hours. They only publish free places during opening hours. You know, that might make sense for drivers but it just makes interpreting the data harder. Since the outage in April 2018 they seem to be open 24/7 and data is published continously. Still, looking closely at some points it becomes hard to determine, for myself at least, if this is real car-in car-out activity. The patterns are so regular at times, e.g. from one weekend to the next, that i find it either creepy or not completely trustworthy. 

In [ ]:
plot_lot_data("ingolstadtreduittilly")

This one's interesting. The number of cars in Ingolstadt seems to be growing. Although, once again, zooming in on the data reveals some strange jumps of the occupied spaces during the night from one week to the next which do not look like a reflection of 3d events. Or could this actually be gradual steps back towards working-life after the first lock-down? 

Changes to the capacity, whether real or digital, do affect the number of free spaces. And i start to realize that it's actually hard work to sample true *car-activity* just from the published number of free spaces. 
 
Gradients do have the same problem. I thought: *lets just look at the **difference** to the previous day* or something like that. This will at least mitigate the *opening-hours problem* and some other automatic or purely digital changes that the free-spaces counter might be subject to. Example:

In [ ]:
df = big_df["freiburgambahnhof"]
df = df[(df.index >= "2020-01-01") & (df.index < "2020-06-01")]

fig = make_subplots(
    rows=4, cols=1,
    vertical_spacing=0.02,
    shared_xaxes=True,
    subplot_titles=[
        "free spaces per hour", "difference to previous hour", 
        "difference to previous day", "difference to previous week"
    ],
)
fig.add_trace(df.plot().data[0], row=1, col=1)
fig.add_trace(df.diff(1).plot().data[0], row=2, col=1)
fig.add_trace(df.diff(24).plot().data[0], row=3, col=1)
fig.add_trace(df.diff(24*7).plot().data[0], row=4, col=1)
fig.update_layout(
    height=1300, showlegend=False, 
    title="'freiburgambahnhof' free spaces and gradients (2020/01 - 2020/05)"
)

One can *see things*, still, it is hard to interpret this data automatically. 

Fine. I'm not a paid scientist, not even a scientist, but i want to scrutinize question #2 a bit: *Has anything in the parking behaviour significantly changed compared to before?* I mean, apart from the fact that there is less parking, anyways. So i'll try to look at the **occupation per hour-of-day**. In my previous parking post i found that there are some hints if occupation is driven by work & shopping activity or by more leisurely demands. 

But first i need to check the opening hours problem. If a lot does list zero free spaces at some point that translates to 1.0 in the `occupied` DataFrame and so i'll simply count the number of times that a lot has full occupation for each hour of day:

In [ ]:
zero_df = pd.concat([
    (occupied[occupied.index.hour == hour] == 1).astype(int).sum()
    for hour in range(24)
], axis=1)
zero_df.columns.rename("hour of day", inplace=True)
zero_df

Lets see. Some Dresden lots seem to be particularily busy during the day but that could also be because of the lot occupation being too small at periods. All the **Lübeck** lots and one in Freiburg do obviously publish zero free spaces when closed, so that's the ones to be careful about when calculating the occupation per hour. Though we also have seen previously that `freiburgambahnhof` did the same until 2018 and `freiburgmartinstor` and `freiburgzaehringertor` do look similar.

Plotting the occupation data of the Lübeck lots hints at another problem:

In [ ]:
df = occupied.loc[:, occupied.columns.map(lambda c: c.startswith("luebeck"))]
(df[(df.index >= "2018-03-01") & (df.index < "2018-03-08")] * 100).round().plot(
    title="Occupation in Lübeck lots (March 2018)",
    labels={"value": "occupation %"}
)

First it looks like the lots open at 6:00 and close at 22:00 but there are these little edges at the corners. It's more likely they open at 6:30 and close at 20:30 or 21:30 but the final value is lost in the average bucketing of 1-hour-steps done in the beginning. Well, if they are closed, their data does not contribute to the *leisure activity* anyways so i'll simply cut off all lots that have a zero-count of more than 400 at conservative times that are safe to assume, before 7:00 and after 20:00

In [ ]:
occupied_open = occupied.copy()
for lot_id in zero_df[zero_df[0] > 400].index:
    df = occupied_open.loc[:, lot_id]
    occupied_open.loc[:, lot_id] = df[(df.index.hour >= 7) & (df.index.hour <= 20)]

Just to make sure i plot the sample range again for all lots:

In [ ]:
df = occupied_open
(df[(df.index >= "2018-03-01") & (df.index < "2018-03-08")] * 100).round().plot(
    title="Occupation during opening times (March 2018)",
    labels={"value": "occupation %"}
)

As far as i can determine, there are no regular hard edges any more. So then gimme that **occupation per hour-of-day** plot, individually for every year:

In [ ]:
def hours_year_group(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df["year"] = df.index.year
    df["hour"] = df.index.hour
    return (
        df.reset_index().set_index(["date", "year", "hour"])
        .unstack("year")
        .groupby(level="hour").mean()
        .groupby(level="year", axis=1).mean()
    )
    
(hours_year_group(occupied_open) * 100).plot(
    title="mean occupation per hour of day",
    labels={"value": "occupation %"},
    color_discrete_sequence=["#aa4", "#4a4", "#4aa", "#48a", "#f00"]
)

Amazing, isn't it? No, not really. And the bump at 20:00 is not making much sense. Let's plot the mean for each city individually: 

In [ ]:
def per_city_plot(occupied_open: pd.DataFrame, title: Optional[str] = None):
    fig = make_subplots(
        rows=len(CITIES), cols=1,
        vertical_spacing=0.02,
        shared_xaxes=True,
        subplot_titles=CITIES,
    )
    for i, city in enumerate(CITIES):
        df = occupied_open.loc[:, occupied_open.columns.map(lambda c: c.startswith(city.lower()))]
        for trace in (hours_year_group(df) * 100).round().plot(
            labels={"value": "occupation %"},
            color_discrete_sequence=["#aa4", "#4a4", "#4aa", "#48a", "#f00"],
        ).data:
            if i != 0:
                trace.showlegend = False
            fig.add_trace(trace, row=i+1, col=1)
    fig.update_layout(
        title=title or "mean occupation per hour of day", height=1000,
    ).show()
    
per_city_plot(occupied_open)

Obviously, **Lübeck** has parking lots that have closed even before 20:00 at some point. Apart from that, the Lübeck plot actually shows something i am looking for: Through working hours the occupation rate is similar to the years before 2020 while the evenings are certainly less occupied. 

**Freiburg** also shows this little peak at 20:00 which is most likely caused by the *closing hours problem* and not by party goers.

**Dresden** shows a different picture. Seems like in 2020 more cars are simply left standing in the garage during the night. Dresden is quite a nice town with a lot of cool places to visit during the night--if there is no emergency decree, that is.

And as seen previously, **Ingolstadt**'s number of parked cars is growing over the years. In 2016 people stayed out longer compared to the other years. 

Okay, well, please be aware! These are all just my assumptions. To proof anything, each parking lot has to be inspected individually. That is not what i want to do in this post. It has already a couple of Megabytes of javascript in it. I'll stick with these average statistics but remember, if the river is half a meter deep on average, that does not mean that the cow is not going to drown when crossing it.

Finally, i'll just repeat the above plot but for two particular weekdays: Wednesday and Sunday.

In [ ]:
per_city_plot(
    occupied_open[occupied_open.index.map(lambda d: d.weekday() == 2)],
    title="mean occupation per hour of day on Wednesdays",
)

In [ ]:
per_city_plot(
    occupied_open[occupied_open.index.map(lambda d: d.weekday() == 6)],
    title="mean occupation per hour of day on Sundays",
)

Thanks for reading! 

Some applause to the *parkenDD* people and, really, don't drink and drive! 